# 01: Shenzhen Data Preparation

From: jodafons 

This notebook will be responsible to create a final spreadsheet with all images paths, hashes and clinical readings.

In [1]:
import pandas as pd
import numpy as np
import os, sys
import glob
import re
import hashlib
import pathlib

In [2]:
def expand_folder( path , extension):
    return glob.glob(path+'/*.'+extension)

In [3]:
def get_md5(path):
    return hashlib.md5(pathlib.Path(path).read_bytes()).hexdigest()

## Create my v1 table

In [4]:
# this is the location of the raw chenzen data
base_data_raw_path = '/home/jodafons/public/bric_data/Shenzhen/raw'
clinical_path = base_data_raw_path + '/clinical'
images_path = base_data_raw_path + '/images'


In [5]:
def prepare_my_table( clinical_path , images_path ):
    
    d = {
      'target'   : [],
      'image_ID' : [],
      'raw_image_path'     : [],
      'raw_image_md5'      : [],
      'age'      : [],
      'sex'      : [],
      'comment'  : [],
    }

    def treat_string( lines ):
        string = ''
        for s in lines:
            string+=s.replace('\n','').replace('\t','')
        return re.sub(' +', ' ', string)
    
    for idx, path in enumerate(expand_folder(clinical_path, 'txt')):
    
        with open(path,'r') as f:
        
            lines = f.readlines()
            sex = True if 'male' in lines[0] else False # 1 for male and 0 for female
            age = int(re.sub('\D', '', lines[0]))
            # get TB by file name (_1.txt is PTB or _0.txt is NTB)
            target = 1 if '_1.txt' in path else 0
        
            filename = path.split('/')[-1]
            image_filename = filename.replace('txt','png')
            #image_path = images_path+('/tb/' if target else '/no_tb/')+image_filename
            image_path = images_path+'/'+image_filename
            d['target'].append(target)
            d['age'].append(age)
            d['sex'].append(sex)
            d['raw_image_path'].append(image_path)
            d['raw_image_md5'].append(get_md5(image_path))
            d['comment'].append(treat_string(lines[1::]))
            d['image_ID'].append(idx)
   
            
    return pd.DataFrame(d)


df = prepare_my_table(clinical_path, images_path)
    

In [6]:
df.head()

,target,image_ID,raw_image_path,raw_image_md5,age,sex,comment
0,0,0,/home/jodafons/public/bric_data/Shenzhen/raw/i...,14a18d4fc7465b66186efb5c9d0d9bea,26,True,normal
1,1,1,/home/jodafons/public/bric_data/Shenzhen/raw/i...,3ba59bfea5469d26e9409d831bc52ab6,65,True,bilateral PTB
2,0,2,/home/jodafons/public/bric_data/Shenzhen/raw/i...,68ebd9afd4b7670e78afb5147115af1d,38,True,normal
3,1,3,/home/jodafons/public/bric_data/Shenzhen/raw/i...,39746a5dc737f0bcbfb8b91b0ecdc54f,36,True,Right PTB
4,1,4,/home/jodafons/public/bric_data/Shenzhen/raw/i...,53a87376a697ae0e9d39b34056fe9f54,21,True,Right PTB


In [7]:
df.to_csv('/home/jodafons/public/bric_data/Shenzhen/raw/Shenzhen_table_from_raw.csv')

In [8]:
df.loc[df.target==1].to_csv('/home/jodafons/public/bric_data/Shenzhen/raw/Shenzhen_table_from_raw_tb_only.csv')

In [9]:
df.loc[df.target==0].to_csv('/home/jodafons/public/bric_data/Shenzhen/raw/Shenzhen_table_from_raw_notb_only.csv')